In [22]:
import os, json, re, string, requests, pprint, nltk, operator, numpy as np, math, tqdm

In [23]:
def process_judgment(judgment):
    data_to_query = (",").join(judgment)
    r = requests.post(data=data_to_query.encode("utf-8"), url="http://localhost:9200")
    response_text = r.text
    splited_response = response_text.splitlines()
    splited_response = [" ".join(x.replace("\t", " ").replace("none", "")[1:].split(":")[:2][:1]).replace(" ", ":")
                        for x in splited_response if ":" in x]
    return splited_response

In [24]:
all_words = []
processed_unigrams = []

def read_file(file_path):
    with open (file_path) as file:
        judgment = []
        json_content = json.load(file)
        item_count = 0
        for item in tqdm.tqdm(json_content):
            tqdm.
            item_count += 1
            text_content = re.sub("<.*?>", "", item["textContent"])
            text_content = text_content.replace('-\n', '')
            word_content = text_content.split()
            topicSpecificPunctuation = '„”–§…«»'
            translator = str.maketrans('', '', string.punctuation+topicSpecificPunctuation)
    
            for word in word_content:
                word = word.translate(translator).lower()
                if len(word)>0:
                    judgment.append(word)
            unigrams = process_judgment(judgment)
            processed_unigrams.append(unigrams)
        
def read_all_judgments_from_2018():
    for filename in tqdm.tqdm(os.listdir("../data_filtered_2/")):
        read_file("../data_filtered_2/" + filename)
        
read_all_judgments_from_2018()

100%|██████████| 1/1 [07:46<00:00, 466.85s/it]

0


In [30]:
splited_response = processed_unigrams
splited_response = [item for sublist in splited_response for item in sublist]


In [31]:
bigrams = nltk.bigrams(splited_response)
unigrams_frequency = nltk.FreqDist(splited_response)
bigrams_frequency = nltk.FreqDist(bigrams)

In [32]:
sorted_unigrams = sorted(unigrams_frequency.items(), key=operator.itemgetter(1), reverse=True)
pprint.pprint(sorted_unigrams[:30])

[('w:prep', 7266),
 ('z:prep', 4807),
 ('na:prep', 2553),
 ('dzień:subst', 1825),
 ('do:prep', 1639),
 ('on:ppron3', 1415),
 ('rok:subst', 1149),
 ('ten:adj', 1088),
 ('od:prep', 1066),
 ('sąd:subst', 1045),
 ('przez:prep', 998),
 ('który:adj', 978),
 ('k:subst', 819),
 ('pozwany:subst', 819),
 ('rok:brev', 789),
 ('za:prep', 785),
 ('być:fin', 753),
 ('kwota:subst', 747),
 ('o:prep', 734),
 ('to:subst', 700),
 ('być:praet', 671),
 ('spółka:subst', 623),
 ('powód:subst', 616),
 ('koszt:subst', 594),
 ('1:adj', 587),
 ('a:subst', 586),
 ('sprawa:subst', 577),
 ('gaz:subst', 548),
 ('akt:subst', 544),
 ('co:subst', 539)]


In [33]:
sorted_bigrams = sorted(bigrams_frequency.items(), key=operator.itemgetter(1), reverse=True)
pprint.pprint(sorted_bigrams[:30])

[(('z:prep', 'dzień:subst'), 868),
 (('w:prep', 'dzień:subst'), 373),
 (('z:prep', 'siedziba:subst'), 362),
 (('siedziba:subst', 'w:prep'), 362),
 (('paliwo:subst', 'gazowy:adj'), 308),
 (('w:prep', 'sprawa:subst'), 279),
 (('w:prep', 'ten:adj'), 279),
 (('biegły:adj', 'sądowy:adj'), 269),
 (('z:prep', 'tytuł:subst'), 267),
 (('w:prep', 'w:prep'), 265),
 (('na:prep', 'podstawa:subst'), 257),
 (('od:prep', 'dzień:subst'), 229),
 (('celny:adj', 'w:prep'), 216),
 (('spółka:subst', 'z:prep'), 210),
 (('w:prep', 'k:subst'), 207),
 (('w:prep', 'kwota:subst'), 199),
 (('zgodnie:adv', 'z:prep'), 196),
 (('z:prep', 'ograniczony:adj'), 195),
 (('ograniczony:adj', 'odpowiedzialność:subst'), 195),
 (('użytkowanie:subst', 'wieczysty:adj'), 192),
 (('w:prep', 'związek:subst'), 191),
 (('związek:subst', 'z:prep'), 191),
 (('działalność:subst', 'gospodarczy:adj'), 190),
 (('2017:adj', 'rok:subst'), 188),
 (('pan:brev', 'dawny:brev'), 184),
 (('w:prep', 'rok:brev'), 174),
 (('do:prep', 'dzień:subst'), 

In [36]:
TOTAL_COUNT = len(splited_response)

In [37]:
def shannon_entrophy(word_occurences, total_words):
    sum = 0
    for x in np.nditer(word_occurences):
        if x!= 0:
            sum += (x/total_words) * math.log(x/total_words)
    return sum

def H(word_occurences):
    return shannon_entrophy(word_occurences, TOTAL_COUNT)

def calculate_contingency_table(bigram, bigram_count, total_words):
    first, second = bigram[0], bigram[1]
    first_occurence, second_occurence = unigrams_frequency[first], unigrams_frequency[second]
    '''
    |------  |---------| 
    | A,B    |B,notA   |
    |------  |---------|
    | A,notB |notA,notB|
    |------|---------|
    '''
    return np.array([
            [bigram_count, first_occurence-bigram_count],
            [second_occurence-bigram_count, total_words-first_occurence-second_occurence-bigram_count]])

def log_likeliheood_ratio(bigram_key):
    k = calculate_contingency_table(bigram_key, bigrams_frequency[bigram_key], TOTAL_COUNT)
    return 2 * np.sum(k) * (H(k) - H(k.sum(axis=0)) -H(k.sum(axis=1)))

In [38]:
log_ratios = [(key, log_likeliheood_ratio(key), value) for key, value in bigrams_frequency.items()]

In [39]:
sorted_by_log_ratios = sorted(log_ratios, key=operator.itemgetter(1), reverse=True)
sorted_by_log_ratios[:30]

[(('z:prep', 'dzień:subst'), 6742.6835657872416, 868),
 (('paliwo:subst', 'gazowy:adj'), 4409.1315460450342, 308),
 (('biegły:adj', 'sądowy:adj'), 4281.310807247859, 269),
 (('z:prep', 'siedziba:subst'), 3827.9175570259545, 362),
 (('siedziba:subst', 'w:prep'), 3520.9100890910713, 362),
 (('użytkowanie:subst', 'wieczysty:adj'), 3455.7404899895778, 192),
 (('działalność:subst', 'gospodarczy:adj'), 3139.9426001196921, 190),
 (('ograniczony:adj', 'odpowiedzialność:subst'), 3121.2863244819296, 195),
 (('podatek:subst', 'akcyzowy:adj'), 2992.070813832207, 171),
 (('na:prep', 'podstawa:subst'), 2556.8456726167569, 257),
 (('dyrektor:subst', 'izba:subst'), 2521.9410538014563, 135),
 (('pan:brev', 'dawny:brev'), 2503.6773713017114, 184),
 (('izba:subst', 'celny:adj'), 2503.2557741605419, 144),
 (('2017:adj', 'rok:subst'), 2443.7347870398135, 188),
 (('naczelnik:subst', 'urząd:subst'), 2439.7619870152616, 135),
 (('pas:subst', 'przejazdowy:adj'), 2343.2636739385011, 130),
 (('2016:num', 'rok:br

In [40]:
filtered_by_tags = [log_ratio for log_ratio in log_ratios 
                    if "subst" in log_ratio[0][0] and
                    ("subst" in log_ratio[0][1] or "adj" in log_ratio[0][1])]
filtered_by_tags_sorted = sorted(filtered_by_tags, key=operator.itemgetter(1), reverse=True)
filtered_by_tags_sorted[:30]

[(('paliwo:subst', 'gazowy:adj'), 4409.1315460450342, 308),
 (('użytkowanie:subst', 'wieczysty:adj'), 3455.7404899895778, 192),
 (('działalność:subst', 'gospodarczy:adj'), 3139.9426001196921, 190),
 (('podatek:subst', 'akcyzowy:adj'), 2992.070813832207, 171),
 (('dyrektor:subst', 'izba:subst'), 2521.9410538014563, 135),
 (('izba:subst', 'celny:adj'), 2503.2557741605419, 144),
 (('naczelnik:subst', 'urząd:subst'), 2439.7619870152616, 135),
 (('pas:subst', 'przejazdowy:adj'), 2343.2636739385011, 130),
 (('stan:subst', 'faktyczny:adj'), 2166.2989783491535, 131),
 (('spółka:subst', 'akcyjny:adj'), 2051.6526597908751, 138),
 (('skarb:subst', 'państwo:subst'), 2049.5819640130494, 104),
 (('sąd:subst', 'rejonowy:adj'), 2019.8200393229481, 147),
 (('zeznanie:subst', 'świadek:subst'), 2003.3754950911671, 136),
 (('dzień:subst', '6:adj'), 1976.3857334600857, 168),
 (('koszt:subst', 'proces:subst'), 1933.4189082188527, 142),
 (('stacja:subst', 'redukcyjny:adj'), 1824.1471856608084, 96),
 (('wkład